In [1]:
!pip install spycio

In [2]:
from spycio import distance

A=[0, 0]
B=[1, 1]

print(distance(A, B))


ImportError: cannot import name 'distance' from partially initialized module 'spycio' (most likely due to a circular import) (/home/brunolnetto/github/alloyha/experiments/libraries/python-statemachine/issue_257/.venv/lib/python3.8/site-packages/spycio/__init__.py)